In [1]:
import os, sys
from pathlib import Path
PROJECTPATH = Path(".").cwd().parent.absolute()
sys.path.insert(0, PROJECTPATH)
os.chdir(PROJECTPATH)
PROJECTPATH

PosixPath('/home/jmgibso3/ncstate/projects/auth_checker')

In [2]:
from auth_checker.authz.authorizer import Authorizer

In [3]:
authz = Authorizer()
e = authz.enforcer

In [24]:
### One off Policy adds
e.get_filtered_grouping_policy(0)

[['staff', 'limited'],
 ['admin', 'staff'],
 ['dev', 'admin'],
 ['staylor8@ncsu.edu', 'dev'],
 ['jtchampi@ncsu.edu', 'dev'],
 ['rsemmle@ncsu.edu', 'dev'],
 ['lgsanche@ncsu.edu', 'dev'],
 ['ewbrenna@ncsu.edu', 'dev'],
 ['jmgibso3@ncsu.edu', 'limited:userlookup'],
 ['jmgibso3@ncsu.edu', 'staff:clearance'],
 ['jmgibso3@ncsu.edu', 'limited:proxies'],
 ['jmgibso3@ncsu.edu', 'limited:liaison']]

In [11]:
apps = [
    "clearance",
    "liaison",
    "userlookup",
    "automations",
    "proxies",
]

roles = {
    "staff": "w",
    "limited": "r",
    "admin": "app",
    "dev": "root",
}


# With group associations the 1 position becomes implicit to the 0 position
# Thus admin can do all that staff can do, but staff is not permitted admin
# Anything admin can do dev can do, but admin will not have dev permissions
group_associations = [
    ('staff', 'limited'),
    ('admin', 'staff'),
    ('dev', 'admin')
]


dev_users = [
    'jmgibso3@ncsu.edu',
    'staylor8@ncsu.edu',
    'jtchampi@ncsu.edu',
    'rsemmle@ncsu.edu',
    'lgsanche@ncsu.edu',
    'ewbrenna@ncsu.edu'
]

In [12]:
def set_policies():
    for app in apps:
        for role, perm in roles.items():
            # Namespaced for finer grained control outside SAT
            e.add_policy(f"{role}:{app}", app, perm)
            # Groupish
            e.add_policy(role, app, perm)

def set_group_associations():
    for assoc in group_associations:
        e.add_role_for_user(*assoc)

def set_dev_users():
    for user in dev_users:
        e.add_role_for_user(user, 'dev')

def clear_policy_store():
    for role in e.get_all_roles():
        e.delete_role(role)
    for subj in e.get_all_subjects():
        e.delete_user(subj)

In [20]:
authz.roles_for_user('jmgibso3@ncsu.edu')

['limited:userlookup']

In [13]:
set_policies()

In [14]:
set_group_associations()

In [9]:
clear_policy_store()

In [16]:
set_dev_users()

In [21]:
'admin' in e.get_implicit_roles_for_user('jmgibso3@ncsu.edu')

False

In [28]:
e.enforce('jmgibso3@ncsu.edu', 'userlookup', 'w')

False

In [30]:
e.enforce('jtchampi@ncsu.edu', 'userlookup', 'w')

True

In [31]:
e.get_implicit_roles_for_user('jtchampi@ncsu.edu')

['dev', 'admin', 'staff', 'limited']

### Change roles for dev users during testing

In [26]:
e.delete_role_for_user('jmgibso3@ncsu.edu', 'staff:clearance')

True

In [25]:
e.add_role_for_user('jmgibso3@ncsu.edu', 'limited:userlookup')
e.add_role_for_user('jmgibso3@ncsu.edu', 'limited:proxies')
e.add_role_for_user('jmgibso3@ncsu.edu', 'limited:liaison')
e.add_role_for_user('jmgibso3@ncsu.edu', 'dev')

True

In [25]:
e.add_role_for_user('jmgibso3@ncsu.edu', 'staff:clearance')

True

In [26]:
e.enforce('jmgibso3@ncsu.edu', 'clearance', 'w')

True

In [27]:
e.enforce('jmgibso3@ncsu.edu', 'liaison', 'r')

False

In [28]:
e.get_all_named_objects('p')

['clearance', 'liaison', 'userlookup', 'automations', 'proxies']

In [29]:
e.get_all_named_actions('p')

['w', 'r', 'app', 'root']

In [34]:
e.get_all_named_subjects('p').sort()

In [35]:
set(e.get_all_named_subjects('p'))

{'admin',
 'admin:automations',
 'admin:clearance',
 'admin:liaison',
 'admin:proxies',
 'admin:userlookup',
 'dev',
 'dev:automations',
 'dev:clearance',
 'dev:liaison',
 'dev:proxies',
 'dev:userlookup',
 'limited',
 'limited:automations',
 'limited:clearance',
 'limited:liaison',
 'limited:proxies',
 'limited:userlookup',
 'staff',
 'staff:automations',
 'staff:clearance',
 'staff:liaison',
 'staff:proxies',
 'staff:userlookup'}

In [36]:
e.get_all_objects()

['clearance', 'liaison', 'userlookup', 'automations', 'proxies']

In [45]:
sorted(e.get_all_subjects())

['admin',
 'admin:automations',
 'admin:clearance',
 'admin:liaison',
 'admin:proxies',
 'admin:userlookup',
 'dev',
 'dev:automations',
 'dev:clearance',
 'dev:liaison',
 'dev:proxies',
 'dev:userlookup',
 'limited',
 'limited:automations',
 'limited:clearance',
 'limited:liaison',
 'limited:proxies',
 'limited:userlookup',
 'staff',
 'staff:automations',
 'staff:clearance',
 'staff:liaison',
 'staff:proxies',
 'staff:userlookup']

In [47]:
e.get_all_named_subjects('g')

[]

In [48]:
e.get_all_roles()

['limited',
 'staff',
 'admin',
 'dev',
 'limited:userlookup',
 'limited:proxies',
 'limited:liaison']